## 确定文本切割的最优策略
在使用基于检索的生成模型（RAG）处理长文本数据时，合理的文本切割策略是提高模型性能和效率的关键。

文本切割策略主要依赖于两个参数：chunksize（块大小）和overlap（重叠）。正确配置这些参数可以显著影响模型的输出质量和处理速度。
   - chunk_size 基于模型的限制(embedding , LLM )
   - 不同Text splitter 的优劣，如何选取
   - 可视化文本切分的效果，供大家切分文本初步参考
      
## 基于模型选取chunk_size 
   - 首先是embedding model， 向量嵌入模型有Max Tokens 的限制，设置的chunk size不可以超过模型支持的最大长度，否则将丢失语义。


不同的embedding model 支持的 Max Tokens都有不同，具体可参考model 排行
其次是LLM model , 大语言模型有Max sequence length的限制，处理知识增强的时候，prompt中召回的文本不可以超出最大长度。
![Alt text]( )
需要根据不同的LLM支持的最大token长度，选取合适的参数
不同的文本切分策略
1: CharacterTextSplitter - 这是最简单的方法。它默认基于字符（默认为""）来分割，并且通过字符的数量来衡量块的长度。
2：RecursiveCharacterTextSplitter - 基于字符列表拆分文本。
3: Document Specific Splitting - 基于不同的文件类型使用不同的切分 (PDF, Python, Markdown)
4: Semantic Splitting - 基于滑动窗口的语义切分
那我们就开始实际看一下不同的textsplitter切分效果如何？




In [ ]:
! pip install langchain

In [2]:
text = "大家好，我是果粒奶优有果粒，欢迎关注我，让我们一起探索AI"

1. CharacterTextSplitter

In [5]:
from langchain_text_splitters import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="",
    chunk_size=5,
    chunk_overlap=1,
    length_function=len,
    is_separator_regex=False,
)


In [ ]:
text_splitter.split_text(text)